In [235]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from itertools import product
!pip install kmodes

In [236]:
persones=pd.read_csv('Accident_people_2021_2022.csv')
persones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17919 entries, 0 to 17918
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    17919 non-null  int64 
 1   File_Num                      17919 non-null  object
 2   District                      17919 non-null  object
 3   Neighbourhood                 17919 non-null  object
 4   Street_name                   17919 non-null  object
 5   Day_name                      17919 non-null  object
 6   year                          17919 non-null  int64 
 7   month                         17919 non-null  int64 
 8   Month_name                    17919 non-null  object
 9   day                           17919 non-null  int64 
 10  Day_shift                     17919 non-null  object
 11  hour                          17919 non-null  int64 
 12  Pedestrian_situation          17919 non-null  object
 13  Vehicle_type    

In [237]:
weather=pd.read_csv('data_meteo_estudifinal_2021_2022.csv')
weather = weather.drop(weather.columns[0], axis=1)
weather.rename(columns={'DATA_LECTURA':'Date'},inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       35040 non-null  object 
 1   03.TM      35040 non-null  float64
 2   06.HRM     35040 non-null  float64
 3   07.PPT24H  35040 non-null  float64
 4   08.HPA     35040 non-null  float64
 5   09.RS24H   35040 non-null  float64
 6   10.VVM10   35040 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.9+ MB


In [238]:
weather.head()

,Date,03.TM,06.HRM,07.PPT24H,08.HPA,09.RS24H,10.VVM10
0,2021-01-01 00:00:00,10.1,57.0,0.0,1002.2,0.0,1.6
1,2021-01-01 00:30:00,9.9,59.0,0.0,1002.0,0.0,2.1
2,2021-01-01 01:00:00,9.6,59.0,0.0,1001.9,0.0,2.2
3,2021-01-01 01:30:00,9.6,59.0,0.0,1001.8,0.0,2.3
4,2021-01-01 02:00:00,9.6,59.0,0.0,1001.6,0.0,2.1


In [239]:
#FROM ALL THE DATA WE ARE JUST GOING TO FOCUS N¡ON THE DRIVERS BECAUSE
#THE PASSANGER OR THE PEDESTRIAN DOESN0T HAVE ANY KIND OF POWER TO NOT BE IN THE SITUATION OR HAVE ANY CONTROL

persones=persones[['Day_shift','Vehicle_type','Gender','Age','Date']]
weather=weather[['Date','03.TM','06.HRM','10.VVM10']]

#JOIN THE DATASETS WITH THE VARIABLE DATE
variables = pd.merge(persones, weather, on='Date', how='left')
variables.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17919 entries, 0 to 17918
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Day_shift     17919 non-null  object 
 1   Vehicle_type  17919 non-null  object 
 2   Gender        17919 non-null  object 
 3   Age           17919 non-null  object 
 4   Date          17919 non-null  object 
 5   03.TM         17919 non-null  float64
 6   06.HRM        17919 non-null  float64
 7   10.VVM10      17919 non-null  float64
dtypes: float64(3), object(5)
memory usage: 1.2+ MB


In [240]:
#add variable quarter and start studying the different variables
variables['Quarter'] = variables['Date'].apply(lambda x: pd.Timestamp(x).quarter)
variables.head()

,Day_shift,Vehicle_type,Gender,Age,Date,03.TM,06.HRM,10.VVM10,Quarter
0,Afternoon,Vehicle without motor,Women,8,2021-04-13 20:00:00,13.8,78.0,0.8,2
1,Morning,Other,Men,58,2021-03-12 12:00:00,13.0,41.0,2.8,1
2,Afternoon,Other,Women,52,2021-10-11 19:00:00,14.9,75.0,2.7,4
3,Afternoon,Vehicle without motor,Men,47,2021-02-06 15:00:00,22.0,77.0,3.3,1
4,Afternoon,Other,Men,4,2021-09-12 17:00:00,10.8,53.0,0.9,3


In [241]:
# LETS TRANSFORM TO CATHEGORIC THE COLUMN EDAT
#/// desconegut(404) -->NOT KNOWN
#transform value desconegut a 404
variables['Age'] = variables['Age'].replace('Desconegut', '404')
def categorize_age(age):
    if int(age) <= 25:
        return '16-25'
    elif int(age) <= 35 and int(age) > 25:
        return '26-35'
    elif int(age) <= 45 and int(age) > 35:
        return '36-45'
    elif int(age) <= 55 and int(age) > 45:
        return '46-55'
    elif age == '404':
        return 'Age not known'
    else:
        return '+56'

# Apply function to transform ages to categories
variables['Age'] = variables['Age'].apply(categorize_age)

x=variables['Age'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['Age'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['16-25' '+56' '46-55' '26-35' '36-45' 'Age not known']
26-35            25.771527
16-25            20.620570
36-45            19.917406
+56              17.489815
46-55            16.111390
Age not known     0.089291
Name: Age, dtype: float64


In [242]:
#x=variables['03.TM'].unique()
# print(np.sort(x)) the data goes from 3.8 to 34.2
def cat_temp(temp):
    if int(temp) <= 13:
        return 'Very low'
    elif int(temp) <= 17 and int(temp) > 13:
        return 'Low'
    elif int(temp) <= 24 and int(temp) > 17:
        return 'Mild'
    else:
        return 'Hot'
variables['03.TM'] = variables['03.TM'].apply(cat_temp)

x=variables['03.TM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['03.TM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Very low' 'Low' 'Mild' 'Hot']
Mild        30.397902
Very low    24.750265
Hot         23.405324
Low         21.446509
Name: 03.TM, dtype: float64


In [243]:
#x=variables['06.HRM'].unique()
#print(np.sort(x)) from 9 percent to 100 percent
def cat_humidity(hum):
    if int(hum) <= 48:
        return 'Low humidity'
    elif int(hum) <= 60 and int(hum) > 48:
        return 'Moderate humidity'
    elif int(hum) <= 75 and int(hum) > 60:
        return 'High humidity'
    else:
        return 'Very high'
variables['06.HRM'] = variables['06.HRM'].apply(cat_humidity)

x=variables['06.HRM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['06.HRM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Very high' 'Low humidity' 'High humidity' 'Moderate humidity']
High humidity        39.092583
Moderate humidity    25.514817
Low humidity         18.907305
Very high            16.485295
Name: 06.HRM, dtype: float64


In [244]:
#x=variables['10.VVM10'].unique()
#print(np.sort(x)) from 0 to 8.7
def cat_wind(wind):
    if int(wind) <= 1:
        return 'Low speed'
    elif int(wind) <= 2 and int(wind) > 1:
        return 'Moderate speed'
    else:
        return 'High speed'
    
variables['10.VVM10'] = variables['10.VVM10'].apply(cat_wind)

x=variables['10.VVM10'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['10.VVM10'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

['Low speed' 'Moderate speed' 'High speed']
Low speed         43.211117
Moderate speed    30.157933
High speed        26.630950
Name: 10.VVM10, dtype: float64


In [245]:
column_names = variables.columns.tolist()
print(column_names)

['Day_shift', 'Vehicle_type', 'Gender', 'Age', 'Date', '03.TM', '06.HRM', '10.VVM10', 'Quarter']


In [246]:
from itertools import combinations
import pandas as pd

# Load the dataset into a Pandas DataFrame (reemplaza 'tu_archivo.csv' con la ruta de tu archivo CSV)
df = variables

# Lista de variables dada
categorical = ['Vehicle_type', 'Gender', 'Age', 'Day_shift', '03.TM']

for n in range(3, 7):
    # Diccionario para almacenar las 10 mejores combinaciones
    mejores_combinaciones = {}
    # Generar todas las combinaciones de 'n' variables
    combinations_list = list(combinations(categorical, n))
    for combination in combinations_list:
        # Count the number of occurrences of each unique combination of values
        counts = df.groupby(list(combination)).size().reset_index(name='count')

        # Get the top 10 combinations of values that have the highest count
        top_counts = counts.sort_values('count', ascending=False).head(10)

        # Almacena las mejores combinaciones en el diccionario si es la mejor hasta ahora
        if combination not in mejores_combinaciones or top_counts['count'].max() > mejores_combinaciones[combination]['count'].max():
            mejores_combinaciones[combination] = top_counts

    # Imprime las 10 mejores combinaciones para 'n' variables
    print(f"Mejores combinaciones para {n} variables:")
    for combination, top_counts in mejores_combinaciones.items():
        print(combination)
        print(top_counts)


Mejores combinaciones para 3 variables:
('Vehicle_type', 'Gender', 'Age')
   Vehicle_type Gender    Age  count
15   Motorcycle    Men  26-35   1803
16   Motorcycle    Men  36-45   1364
17   Motorcycle    Men  46-55   1159
14   Motorcycle    Men  16-25   1116
22   Motorcycle  Women  26-35    993
13   Motorcycle    Men    +56    830
34        Other  Women    +56    710
21   Motorcycle  Women  16-25    687
23   Motorcycle  Women  36-45    647
24   Motorcycle  Women  46-55    492
('Vehicle_type', 'Gender', 'Day_shift')
   Vehicle_type Gender  Day_shift  count
7    Motorcycle    Men  Afternoon   3300
8    Motorcycle    Men    Morning   2276
12   Motorcycle  Women  Afternoon   1692
13   Motorcycle  Women    Morning   1186
19        Other  Women  Afternoon    989
4           Car  Women  Afternoon    915
15        Other    Men  Afternoon    881
0           Car    Men  Afternoon    844
20        Other  Women    Morning    740
16        Other    Men    Morning    727
('Vehicle_type', 'Gender', '

In [250]:
from itertools import combinations
import pandas as pd

# Load the dataset into a Pandas DataFrame (reemplaza 'tu_archivo.csv' con la ruta de tu archivo CSV)
df = variables

# Lista de variables dada
categorical = ['Vehicle_type', 'Gender', 'Age', 'Day_shift', '03.TM']

for n in range(3, 7):
    # Diccionario para almacenar las 10 mejores combinaciones
    mejores_combinaciones = {}
    peor_mejor_combinacion = 1.0  # Inicializado a un valor alto
    # Generar todas las combinaciones de 'n' variables
    combinations_list = list(combinations(categorical, n))
    for combination in combinations_list:
        # Count the number of occurrences of each unique combination of values
        counts = df.groupby(list(combination)).size().reset_index(name='count')

        # Get the top 10 combinations of values that are the most repeated in the dataset
        top_counts = counts.sort_values('count', ascending=False).head(10)

        # Almacena las mejores combinaciones en el diccionario
        mejores_combinaciones[combination] = top_counts

    # Imprime las mejores combinaciones para 'n' variables
    top_10_combinations = sorted(mejores_combinaciones.items(), key=lambda x: x[1]['count'].max(), reverse=True)[:10]
    
    print(top_10_combinations)

[(('Vehicle_type', 'Gender', 'Day_shift'),    Vehicle_type Gender  Day_shift  count
7    Motorcycle    Men  Afternoon   3300
8    Motorcycle    Men    Morning   2276
12   Motorcycle  Women  Afternoon   1692
13   Motorcycle  Women    Morning   1186
19        Other  Women  Afternoon    989
4           Car  Women  Afternoon    915
15        Other    Men  Afternoon    881
0           Car    Men  Afternoon    844
20        Other  Women    Morning    740
16        Other    Men    Morning    727), (('Vehicle_type', 'Gender', '03.TM'),    Vehicle_type Gender     03.TM  count
11   Motorcycle    Men      Mild   1879
12   Motorcycle    Men  Very low   1570
9    Motorcycle    Men       Hot   1447
10   Motorcycle    Men       Low   1378
16   Motorcycle  Women      Mild    950
17   Motorcycle  Women  Very low    851
14   Motorcycle  Women       Hot    715
15   Motorcycle  Women       Low    649
25        Other  Women      Mild    592
20        Other    Men      Mild    563), (('Vehicle_type', 'Gende

In [249]:
mejores_combinaciones

{}